In [2]:
import pandas as pd
import sqlite3
import os
from pathlib import Path

# Define paths
excel_folder = Path('ibb-secim')
db_path = 'ibb_secim.db'

# Get all Excel files
excel_files = list(excel_folder.glob('*.xlsx'))
print(f"Found {len(excel_files)} Excel files:")
for file in excel_files:
    print(f"  - {file.name}")

# Create SQLite connection
conn = sqlite3.connect(db_path)

# Read and import each Excel file
for excel_file in excel_files:
    # Get table name from filename (remove .xlsx and clean it)
    table_name = excel_file.stem.replace('-', '_').replace(' ', '_')
    
    print(f"\nProcessing {excel_file.name}...")
    
    try:
        # Read Excel file
        df = pd.read_excel(excel_file)
        
        print(f"  Rows: {len(df)}, Columns: {len(df.columns)}")
        print(f"  Column names: {list(df.columns)}")
        
        # Write to SQLite
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"  ✓ Successfully imported to table '{table_name}'")
        
    except Exception as e:
        print(f"  ✗ Error processing {excel_file.name}: {str(e)}")

# Commit and close
conn.commit()

# Display summary
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print(f"\n{'='*50}")
print(f"Database created successfully: {db_path}")
print(f"Total tables: {len(tables)}")
print("\nTables in database:")
for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
    count = cursor.fetchone()[0]
    print(f"  - {table[0]}: {count} rows")

conn.close()
print(f"\n{'='*50}")
print("Database connection closed.")

Found 8 Excel files:
  - bagcilar-ibb.xlsx
  - bakırkoy-ibb.xlsx
  - besiktas-ibb.xlsx
  - beykoz-ibb.xlsx
  - esenyurt-ibb.xlsx
  - fatih-ibb.xlsx
  - kadıkoy-ibb.xlsx
  - uskudar-ibb.xlsx

Processing bagcilar-ibb.xlsx...
  Rows: 1453, Columns: 15
  Column names: ['Sıra No', 'İl Adı', 'İlçe Adı', 'Mahalle/Köy', 'Sandık No', 'Kayıtlı Seçmen Sayısı', 'Oy Kullanan Seçmen Sayısı', 'İtirazsız Geçerli Oy Sayısı', 'İtirazlı Geçerli Oy Sayısı', 'Toplam Geçerli Oy', 'Toplam Geçersiz Oy', 'SAADET', 'VATAN PARTİSİ', 'CHP', 'AK PARTİ']
  ✓ Successfully imported to table 'bagcilar_ibb'

Processing bakırkoy-ibb.xlsx...
  Rows: 501, Columns: 15
  Column names: ['Sıra No', 'İl Adı', 'İlçe Adı', 'Mahalle/Köy', 'Sandık No', 'Kayıtlı Seçmen Sayısı', 'Oy Kullanan Seçmen Sayısı', 'İtirazsız Geçerli Oy Sayısı', 'İtirazlı Geçerli Oy Sayısı', 'Toplam Geçerli Oy', 'Toplam Geçersiz Oy', 'SAADET', 'VATAN PARTİSİ', 'CHP', 'AK PARTİ']
  ✓ Successfully imported to table 'bakırkoy_ibb'

Processing besiktas-ibb.xlsx

In [3]:
# Extract unique İlçe and Mahalle/Köy names from all tables
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('ibb_secim.db')

# Get all table names
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]

print("Tables found:", tables)
print("\n" + "="*70)

# Let's first check the structure of one table to understand the column names
sample_table = tables[0]
sample_df = pd.read_sql(f"SELECT * FROM {sample_table} LIMIT 5", conn)
print(f"\nSample data from '{sample_table}':")
print(sample_df.head())
print(f"\nColumn names: {list(sample_df.columns)}")

Tables found: ['bagcilar_ibb', 'bakırkoy_ibb', 'besiktas_ibb', 'beykoz_ibb', 'esenyurt_ibb', 'fatih_ibb', 'kadıkoy_ibb', 'uskudar_ibb']


Sample data from 'bagcilar_ibb':
   Sıra No    İl Adı  İlçe Adı Mahalle/Köy  Sandık No  Kayıtlı Seçmen Sayısı  \
0        1  İSTANBUL  BAĞCILAR  ÇINAR MAH.       1001                    344   
1        2  İSTANBUL  BAĞCILAR  ÇINAR MAH.       1002                    344   
2        3  İSTANBUL  BAĞCILAR  ÇINAR MAH.       1003                    344   
3        4  İSTANBUL  BAĞCILAR  ÇINAR MAH.       1004                    344   
4        5  İSTANBUL  BAĞCILAR  ÇINAR MAH.       1005                    344   

   Oy Kullanan Seçmen Sayısı  İtirazsız Geçerli Oy Sayısı  \
0                        246                          235   
1                        254                          248   
2                        287                          280   
3                        284                          279   
4                        269               

In [4]:
# Extract all unique İlçe and Mahalle/Köy combinations from all tables
all_ilce = []
all_mahalle = []
ilce_mahalle_combinations = []

for table in tables:
    df = pd.read_sql(f"SELECT DISTINCT `İlçe Adı`, `Mahalle/Köy` FROM {table}", conn)
    all_ilce.extend(df['İlçe Adı'].tolist())
    all_mahalle.extend(df['Mahalle/Köy'].tolist())
    
    # Create combinations (İlçe - Mahalle pairs)
    for _, row in df.iterrows():
        ilce_mahalle_combinations.append({
            'İlçe': row['İlçe Adı'],
            'Mahalle/Köy': row['Mahalle/Köy']
        })

# Get unique values
unique_ilce = sorted(list(set(all_ilce)))
unique_mahalle = sorted(list(set(all_mahalle)))

print("="*70)
print(f"UNIQUE İLÇE (DISTRICTS) - Total: {len(unique_ilce)}")
print("="*70)
for ilce in unique_ilce:
    print(f"  • {ilce}")

print("\n" + "="*70)
print(f"UNIQUE MAHALLE/KÖY (NEIGHBORHOODS/VILLAGES) - Total: {len(unique_mahalle)}")
print("="*70)
for mahalle in unique_mahalle[:20]:  # Show first 20
    print(f"  • {mahalle}")
if len(unique_mahalle) > 20:
    print(f"  ... and {len(unique_mahalle) - 20} more")

# Create a DataFrame with unique combinations
df_combinations = pd.DataFrame(ilce_mahalle_combinations).drop_duplicates().sort_values(['İlçe', 'Mahalle/Köy'])
print("\n" + "="*70)
print(f"UNIQUE İLÇE-MAHALLE COMBINATIONS - Total: {len(df_combinations)}")
print("="*70)
print(df_combinations.head(20))

conn.close()

UNIQUE İLÇE (DISTRICTS) - Total: 8
  • BAKIRKÖY
  • BAĞCILAR
  • BEYKOZ
  • BEŞİKTAŞ
  • ESENYURT
  • FATİH
  • KADIKÖY
  • ÜSKÜDAR

UNIQUE MAHALLE/KÖY (NEIGHBORHOODS/VILLAGES) - Total: 244
  • 100. YIL MAH.
  • 15 TEMMUZ MAH.
  • 19 MAYIS MAH.
  • ABBASAĞA MAH.
  • ACARLAR MAH.
  • ACIBADEM MAH.
  • AHMEDİYE MAH.
  • AKAT MAH.
  • AKBABA MAH.
  • AKEVLER MAH.
  • AKSARAY MAH.
  • AKÇABURGAZ MAH.
  • AKŞEMSEDDİN MAH.
  • AKŞEMSETTİN MAH.
  • ALEMDAR MAH.
  • ALTUNİZADE MAH.
  • ALİ KUŞÇU MAH.
  • ALİBAHADIR MAH.
  • ANADOLU HİSARI MAH.
  • ANADOLU KAVAĞI MAH.
  ... and 224 more

UNIQUE İLÇE-MAHALLE COMBINATIONS - Total: 261
        İlçe                              Mahalle/Köy
22  BAKIRKÖY                     ATAKÖY 1. KISIM MAH.
23  BAKIRKÖY                 ATAKÖY 2-5-6. KISIM MAH.
24  BAKIRKÖY                ATAKÖY 3-4-11. KISIM MAH.
25  BAKIRKÖY              ATAKÖY 7-8-9-10. KISIM MAH.
37  BAKIRKÖY  BAKIRKÖY KADIN KAPALI CEZA İNFAZ KURUMU
26  BAKIRKÖY                            BASI

In [5]:
# Save the unique lists to CSV files for Google Maps API usage
import json

# Save İlçe list
df_ilce = pd.DataFrame({'İlçe': unique_ilce})
df_ilce.to_csv('unique_ilce.csv', index=False, encoding='utf-8-sig')
print("✓ Saved unique_ilce.csv")

# Save Mahalle/Köy list
df_mahalle = pd.DataFrame({'Mahalle/Köy': unique_mahalle})
df_mahalle.to_csv('unique_mahalle.csv', index=False, encoding='utf-8-sig')
print("✓ Saved unique_mahalle.csv")

# Save İlçe-Mahalle combinations
df_combinations.to_csv('ilce_mahalle_combinations.csv', index=False, encoding='utf-8-sig')
print("✓ Saved ilce_mahalle_combinations.csv")

# Also save as JSON for easier API integration
ilce_json = {'ilce_list': unique_ilce}
with open('unique_ilce.json', 'w', encoding='utf-8') as f:
    json.dump(ilce_json, f, ensure_ascii=False, indent=2)
print("✓ Saved unique_ilce.json")

mahalle_json = {'mahalle_list': unique_mahalle}
with open('unique_mahalle.json', 'w', encoding='utf-8') as f:
    json.dump(mahalle_json, f, ensure_ascii=False, indent=2)
print("✓ Saved unique_mahalle.json")

combinations_json = df_combinations.to_dict('records')
with open('ilce_mahalle_combinations.json', 'w', encoding='utf-8') as f:
    json.dump(combinations_json, f, ensure_ascii=False, indent=2)
print("✓ Saved ilce_mahalle_combinations.json")

print("\n" + "="*70)
print("SUMMARY FOR GOOGLE MAPS API:")
print("="*70)
print(f"Total Districts (İlçe): {len(unique_ilce)}")
print(f"Total Neighborhoods (Mahalle/Köy): {len(unique_mahalle)}")
print(f"Total İlçe-Mahalle Combinations: {len(df_combinations)}")
print("\nFiles created:")
print("  • unique_ilce.csv / unique_ilce.json")
print("  • unique_mahalle.csv / unique_mahalle.json")
print("  • ilce_mahalle_combinations.csv / ilce_mahalle_combinations.json")
print("\nFor Google Maps API queries, use format:")
print("  '{Mahalle}, {İlçe}, İstanbul, Turkey'")
print("\nExample:")
print(f"  '{df_combinations.iloc[0]['Mahalle/Köy']}, {df_combinations.iloc[0]['İlçe']}, İstanbul, Turkey'")

✓ Saved unique_ilce.csv
✓ Saved unique_mahalle.csv
✓ Saved ilce_mahalle_combinations.csv
✓ Saved unique_ilce.json
✓ Saved unique_mahalle.json
✓ Saved ilce_mahalle_combinations.json

SUMMARY FOR GOOGLE MAPS API:
Total Districts (İlçe): 8
Total Neighborhoods (Mahalle/Köy): 244
Total İlçe-Mahalle Combinations: 261

Files created:
  • unique_ilce.csv / unique_ilce.json
  • unique_mahalle.csv / unique_mahalle.json
  • ilce_mahalle_combinations.csv / ilce_mahalle_combinations.json

For Google Maps API queries, use format:
  '{Mahalle}, {İlçe}, İstanbul, Turkey'

Example:
  'ATAKÖY 1. KISIM MAH., BAKIRKÖY, İstanbul, Turkey'


In [15]:
%pip install shapely

Note: you may need to restart the kernel to use updated packages.


In [20]:
import os
import json
import requests
from dotenv import load_dotenv
import time
from shapely.geometry import shape, Point
from datetime import datetime

# Load environment variables
load_dotenv()
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API")

# Target districts (exact names from GeoJSON)
TARGET_DISTRICTS = [
    "Bağcılar", "Bakırköy", "Beşiktaş", "Beykoz", 
    "Esenyurt", "Fatih", "Kadıköy", "Üsküdar"
]

# Place types to search for
PLACE_TYPES = [
    "restaurant", "library", "school", "park", "atm", "cafe",
    "pharmacy", "hospital", "mosque", "bus_station", 
    "train_station", "transit_station"
]

def load_mahalle_geojson():
    """Load mahalle GeoJSON and filter for target districts"""
    print("📂 Loading mahalle_geojson.json...")
    
    with open('mahalle_geojson.json', 'r', encoding='utf-8') as f:
        geojson_data = json.load(f)
    
    filtered_features = []
    
    for feature in geojson_data['features']:
        address = feature['properties'].get('address', {})
        district = address.get('town', '')
        
        if district in TARGET_DISTRICTS:
            filtered_features.append(feature)
    
    print(f"✅ Found {len(filtered_features)} mahalles in target districts")
    
    # Show distribution by district
    district_counts = {}
    for feature in filtered_features:
        district = feature['properties']['address'].get('town', 'Unknown')
        district_counts[district] = district_counts.get(district, 0) + 1
    
    print("\n📊 Distribution by district:")
    for district, count in sorted(district_counts.items()):
        print(f"  • {district}: {count} mahalles")
    
    return filtered_features

def get_polygon_bounds(geometry):
    """Get the bounding box center of a polygon"""
    poly = shape(geometry)
    centroid = poly.centroid
    return centroid.y, centroid.x  # lat, lng

def count_places_in_polygon(geometry, place_type):
    """Count places within a polygon using centroid-based search"""
    poly = shape(geometry)
    centroid = poly.centroid
    lat, lng = centroid.y, centroid.x
    
    # Calculate approximate radius from polygon bounds
    bounds = poly.bounds  # (minx, miny, maxx, maxy)
    width = (bounds[2] - bounds[0]) * 111000  # Convert to meters (approx)
    height = (bounds[3] - bounds[1]) * 111000
    radius = int(max(width, height) / 2)
    radius = min(radius, 5000)  # Cap at 5km
    radius = max(radius, 500)   # Minimum 500m
    
    # Search for places
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": place_type,
        "key": GOOGLE_MAPS_API_KEY
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        
        if data["status"] == "OK":
            # Filter results to only include those actually inside the polygon
            count = 0
            for result in data["results"]:
                place_lat = result["geometry"]["location"]["lat"]
                place_lng = result["geometry"]["location"]["lng"]
                point = Point(place_lng, place_lat)
                
                if poly.contains(point):
                    count += 1
            
            return count
        elif data["status"] == "ZERO_RESULTS":
            return 0
        else:
            return 0
            
    except Exception as e:
        return 0

def process_mahalle(feature, index, total):
    """Process one mahalle and count all place types"""
    properties = feature['properties']
    geometry = feature['geometry']
    address = properties.get('address', {})
    
    # Extract mahalle and district names correctly
    mahalle_name = address.get('suburb', address.get('neighbourhood', 'Unknown'))
    district_name = address.get('town', 'Unknown')
    
    print(f"\n[{index+1}/{total}] 🏘️ Processing: {mahalle_name}, {district_name}")
    
    # Get polygon center
    lat, lng = get_polygon_bounds(geometry)
    print(f"  📍 Centroid: ({lat:.6f}, {lng:.6f})")
    
    result = {
        "mahalle": mahalle_name,
        "district": district_name,
        "centroid_lat": lat,
        "centroid_lng": lng,
        "places": {}
    }
    
    # Count each place type
    for place_type in PLACE_TYPES:
        count = count_places_in_polygon(geometry, place_type)
        result["places"][place_type] = count
        print(f"    {place_type}: {count}", end=", " if place_type != PLACE_TYPES[-1] else "\n")
        time.sleep(0.2)  # Rate limiting
    
    return result

# PROCESS ALL MAHALLES
print("=" * 70)
print("🚀 PROCESSING ALL MAHALLES IN TARGET DISTRICTS")
print("=" * 70)

# Load filtered mahalles
mahalles = load_mahalle_geojson()

if not mahalles:
    print("❌ No mahalles found for target districts!")
else:
    print(f"\n⏰ Starting at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📊 Total mahalles to process: {len(mahalles)}")
    print(f"⏱️ Estimated time: ~{len(mahalles) * len(PLACE_TYPES) * 0.2 / 60:.1f} minutes")
    print("\n" + "=" * 70)
    
    all_results = []
    failed_count = 0
    
    for idx, mahalle in enumerate(mahalles):
        try:
            result = process_mahalle(mahalle, idx, len(mahalles))
            if result:
                all_results.append(result)
            else:
                failed_count += 1
        except Exception as e:
            print(f"  ❌ Error: {e}")
            failed_count += 1
            continue
    
    print("\n" + "=" * 70)
    print("✅ PROCESSING COMPLETE!")
    print("=" * 70)
    print(f"⏰ Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"✓ Successfully processed: {len(all_results)} mahalles")
    print(f"✗ Failed: {failed_count} mahalles")
    
    # Save results
    output_data = {
        "metadata": {
            "total_mahalles": len(all_results),
            "districts": TARGET_DISTRICTS,
            "place_types": PLACE_TYPES,
            "timestamp": datetime.now().isoformat()
        },
        "results": all_results
    }
    
    output_filename = f"mahalle_places_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(output_data, f, indent=2, ensure_ascii=False)
    
    print(f"\n💾 Saved to: {output_filename}")
    
    # Summary statistics by district
    print("\n" + "=" * 70)
    print("📊 SUMMARY BY DISTRICT:")
    print("=" * 70)
    
    district_summary = {}
    for result in all_results:
        district = result['district']
        if district not in district_summary:
            district_summary[district] = {
                'count': 0,
                'total_places': {ptype: 0 for ptype in PLACE_TYPES}
            }
        
        district_summary[district]['count'] += 1
        for ptype in PLACE_TYPES:
            district_summary[district]['total_places'][ptype] += result['places'].get(ptype, 0)
    
    for district in sorted(district_summary.keys()):
        summary = district_summary[district]
        total = sum(summary['total_places'].values())
        print(f"\n{district}: {summary['count']} mahalles, {total} total places")
        for ptype, count in sorted(summary['total_places'].items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"  • {ptype}: {count}")
    
    print("\n" + "=" * 70)
    print("🎉 ALL DONE!")
    print("=" * 70)

🚀 PROCESSING ALL MAHALLES IN TARGET DISTRICTS
📂 Loading mahalle_geojson.json...
✅ Found 164 mahalles in target districts

📊 Distribution by district:
  • Bakırköy: 15 mahalles
  • Bağcılar: 15 mahalles
  • Beykoz: 16 mahalles
  • Beşiktaş: 11 mahalles
  • Esenyurt: 5 mahalles
  • Fatih: 57 mahalles
  • Kadıköy: 12 mahalles
  • Üsküdar: 33 mahalles

⏰ Starting at: 2025-10-25 02:06:18
📊 Total mahalles to process: 164
⏱️ Estimated time: ~6.6 minutes


[1/164] 🏘️ Processing: Balmumcu Mahallesi, Beşiktaş
  📍 Centroid: (41.059527, 29.015073)
    restaurant: 0,     library: 0,     school: 13,     park: 2,     atm: 5,     cafe: 4,     pharmacy: 3,     hospital: 2,     mosque: 1,     bus_station: 1,     train_station: 0,     transit_station: 5

[2/164] 🏘️ Processing: Bebek Mahallesi, Beşiktaş
  📍 Centroid: (41.078970, 29.043979)
    restaurant: 11,     library: 1,     school: 4,     park: 12,     atm: 13,     cafe: 16,     pharmacy: 6,     hospital: 1,     mosque: 4,     bus_station: 0,     tra

In [24]:
URL_LIST = ["https://data.ibb.gov.tr/dataset/61f39231-b963-469d-bdd4-d3d430965122/resource/a6266dec-e871-4aa6-85a6-1d5cbe019b8b/download/index_yasam_kalitesi.geojson",
            "https://data.ibb.gov.tr/dataset/5f441b9f-348e-465c-9ad8-204e5e841be4/resource/f8fb5fd3-ae73-464e-8ab0-78184ae969ef/download/index_yurunebilirlik.geojson",
            "https://data.ibb.gov.tr/dataset/a86fd366-e913-4940-b168-4cccb48603fc/resource/60bb9c23-fef7-4099-bc18-3cfef1feff69/download/kulturel_aktivite_index.geojson"]

In [30]:
import requests
import json
from collections import defaultdict

print("="*70)
print("📥 DOWNLOADING AND MERGING GEOJSON FILES")
print("="*70)

# Download all GeoJSON files
all_geojson_data = []

for idx, url in enumerate(URL_LIST, 1):
    print(f"\n[{idx}/{len(URL_LIST)}] Downloading from:")
    print(f"  {url[:80]}...")
    
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        # Extract the dataset name from URL for identification
        if "yasam_kalitesi" in url:
            dataset_name = "yasam_kalitesi"
        elif "yurunebilirlik" in url:
            dataset_name = "yurunebilirlik"
        elif "kulturel_aktivite" in url:
            dataset_name = "kulturel_aktivite"
        else:
            dataset_name = f"dataset_{idx}"
        
        print(f"  ✅ Downloaded: {dataset_name}")
        print(f"  Features count: {len(data.get('features', []))}")
        
        all_geojson_data.append({
            'name': dataset_name,
            'data': data
        })
        
    except Exception as e:
        print(f"  ❌ Error downloading: {e}")
        continue

print("\n" + "="*70)
print("🔄 MERGING DATA BY İLÇE AND MAHALLE")
print("="*70)

# Create a dictionary to merge data by ilçe and mahalle
merged_data = defaultdict(lambda: {
    'ilce': None,
    'mahalle': None,
    'geometry': None,
    'datasets': {}
})

# Process each dataset
for dataset_info in all_geojson_data:
    dataset_name = dataset_info['name']
    features = dataset_info['data'].get('features', [])
    
    print(f"\n📊 Processing {dataset_name}...")
    
    for feature in features:
        props = feature.get('properties', {})
        
        # Extract ilçe and mahalle (try different possible field names)
        ilce = props.get('ilce_adi') or props.get('ilce') or props.get('ILCE_ADI')
        mahalle = props.get('mahalle_adi') or props.get('mahalle') or props.get('MAHALLE_ADI')
        
        if not ilce or not mahalle:
            continue
        
        # Create unique key
        key = f"{ilce}_{mahalle}"
        
        # Store basic info if not already stored
        if merged_data[key]['ilce'] is None:
            merged_data[key]['ilce'] = ilce
            merged_data[key]['mahalle'] = mahalle
            merged_data[key]['geometry'] = feature.get('geometry')
        
        # Add dataset-specific properties
        merged_data[key]['datasets'][dataset_name] = props

print(f"\n✅ Merged into {len(merged_data)} unique İlçe-Mahalle combinations")

# Convert to structured format with flattened properties
output_data = {
    'type': 'FeatureCollection',
    'metadata': {
        'total_locations': len(merged_data),
        'datasets_merged': [d['name'] for d in all_geojson_data],
        'description': 'Merged GeoJSON data by İlçe and Mahalle'
    },
    'features': []
}

for key, data in merged_data.items():
    # Build flattened properties
    properties = {
        'ilce': data['ilce'],
        'mahalle': data['mahalle']
    }
    
    # Extract key fields from each dataset
    if 'yasam_kalitesi' in data['datasets']:
        yk_props = data['datasets']['yasam_kalitesi']
        properties['INDEX_YASAM_KALITESI'] = yk_props.get('INDEX_YASAM_KALITESI')
    
    if 'yurunebilirlik' in data['datasets']:
        yr_props = data['datasets']['yurunebilirlik']
        properties['INDEX_YURUNEBILIRLIK'] = yr_props.get('INDEX_YURUNEBILIRLIK')
    
    if 'kulturel_aktivite' in data['datasets']:
        ka_props = data['datasets']['kulturel_aktivite']
        properties['KULTUREL_AKTIVITE_INDEX'] = ka_props.get('KULTUREL_AKTIVITE_INDEX')
    
    feature = {
        'type': 'Feature',
        'properties': properties,
        'geometry': data['geometry']
    }
    output_data['features'].append(feature)

# Save merged data
output_filename = 'merged_ibb_indices.json'
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"\n💾 Saved to: {output_filename}")

# Show summary by İlçe
print("\n" + "="*70)
print("📊 SUMMARY BY İLÇE:")
print("="*70)

ilce_counts = defaultdict(int)
for data in merged_data.values():
    ilce_counts[data['ilce']] += 1

for ilce, count in sorted(ilce_counts.items()):
    print(f"  • {ilce}: {count} mahalles")

print("\n" + "="*70)
print("✅ MERGE COMPLETE!")
print("="*70)

# Show sample data
print("\n📋 Sample merged entry:")
sample_key = list(merged_data.keys())[0]
sample_data = merged_data[sample_key]
print(f"\nİlçe: {sample_data['ilce']}")
print(f"Mahalle: {sample_data['mahalle']}")
print(f"Datasets included:")
for dataset_name in sample_data['datasets'].keys():
    print(f"  • {dataset_name}")

# Show flattened properties from output
if output_data['features']:
    print(f"\n📊 Sample flattened properties:")
    sample_feature = output_data['features'][0]
    for key, value in sample_feature['properties'].items():
        print(f"  {key}: {value}")

📥 DOWNLOADING AND MERGING GEOJSON FILES

[1/3] Downloading from:
  https://data.ibb.gov.tr/dataset/61f39231-b963-469d-bdd4-d3d430965122/resource/a6...
  ✅ Downloaded: yasam_kalitesi
  Features count: 901

[2/3] Downloading from:
  https://data.ibb.gov.tr/dataset/5f441b9f-348e-465c-9ad8-204e5e841be4/resource/f8...
  ✅ Downloaded: yasam_kalitesi
  Features count: 901

[2/3] Downloading from:
  https://data.ibb.gov.tr/dataset/5f441b9f-348e-465c-9ad8-204e5e841be4/resource/f8...
  ✅ Downloaded: yurunebilirlik
  Features count: 901

[3/3] Downloading from:
  https://data.ibb.gov.tr/dataset/a86fd366-e913-4940-b168-4cccb48603fc/resource/60...
  ✅ Downloaded: yurunebilirlik
  Features count: 901

[3/3] Downloading from:
  https://data.ibb.gov.tr/dataset/a86fd366-e913-4940-b168-4cccb48603fc/resource/60...
  ✅ Downloaded: kulturel_aktivite
  Features count: 901

🔄 MERGING DATA BY İLÇE AND MAHALLE

📊 Processing yasam_kalitesi...

📊 Processing yurunebilirlik...

📊 Processing kulturel_aktivite...

✅

In [26]:
import sqlite3
import pandas as pd

print("="*70)
print("📊 IBB_SECIM.DB - DATABASE SCHEMA")
print("="*70)

# Connect to the database
conn = sqlite3.connect('ibb_secim.db')
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = [table[0] for table in cursor.fetchall()]

print(f"\n📁 Total Tables: {len(tables)}\n")

# For each table, show its schema
for idx, table_name in enumerate(tables, 1):
    print("="*70)
    print(f"[{idx}/{len(tables)}] TABLE: {table_name}")
    print("="*70)
    
    # Get table schema using PRAGMA
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    
    # Get row count
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_count = cursor.fetchone()[0]
    
    print(f"\n📈 Total Rows: {row_count}")
    print(f"\n📋 Columns ({len(columns)}):")
    print("-"*70)
    
    # Column headers
    print(f"{'#':<4} {'Column Name':<30} {'Type':<15} {'Nullable':<10}")
    print("-"*70)
    
    for col in columns:
        col_id = col[0]
        col_name = col[1]
        col_type = col[2]
        not_null = "NOT NULL" if col[3] else "NULL"
        
        print(f"{col_id:<4} {col_name:<30} {col_type:<15} {not_null:<10}")
    
    # Show sample data (first 3 rows)
    print(f"\n📄 Sample Data (first 3 rows):")
    print("-"*70)
    
    try:
        df_sample = pd.read_sql(f"SELECT * FROM {table_name} LIMIT 3", conn)
        print(df_sample.to_string(index=False))
    except Exception as e:
        print(f"  Error reading sample data: {e}")
    
    print("\n")

conn.close()

print("="*70)
print("✅ SCHEMA INSPECTION COMPLETE")
print("="*70)

📊 IBB_SECIM.DB - DATABASE SCHEMA

📁 Total Tables: 8

[1/8] TABLE: bagcilar_ibb

📈 Total Rows: 1453

📋 Columns (15):
----------------------------------------------------------------------
#    Column Name                    Type            Nullable  
----------------------------------------------------------------------
0    Sıra No                        INTEGER         NULL      
1    İl Adı                         TEXT            NULL      
2    İlçe Adı                       TEXT            NULL      
3    Mahalle/Köy                    TEXT            NULL      
4    Sandık No                      INTEGER         NULL      
5    Kayıtlı Seçmen Sayısı          INTEGER         NULL      
6    Oy Kullanan Seçmen Sayısı      INTEGER         NULL      
7    İtirazsız Geçerli Oy Sayısı    INTEGER         NULL      
8    İtirazlı Geçerli Oy Sayısı     INTEGER         NULL      
9    Toplam Geçerli Oy              INTEGER         NULL      
10   Toplam Geçersiz Oy             INTEGER      

In [31]:
import json
import pandas as pd

print("="*70)
print("📄 EXPORTING MERGED DATA TO CSV")
print("="*70)

# Load the merged GeoJSON file
with open('merged_ibb_indices.json', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

print(f"\n✅ Loaded: {geojson_data['metadata']['total_locations']} locations")

# Extract features and convert to DataFrame
rows = []
for feature in geojson_data['features']:
    props = feature['properties']
    
    # Create a row with all properties
    row = {
        'ilce': props.get('ilce'),
        'mahalle': props.get('mahalle'),
        'INDEX_YASAM_KALITESI': props.get('INDEX_YASAM_KALITESI'),
        'INDEX_YURUNEBILIRLIK': props.get('INDEX_YURUNEBILIRLIK'),
        'KULTUREL_AKTIVITE_INDEX': props.get('KULTUREL_AKTIVITE_INDEX')
    }
    
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

print(f"\n📊 DataFrame created:")
print(f"  Rows: {len(df)}")
print(f"  Columns: {list(df.columns)}")

# Show sample data
print("\n📋 Sample data (first 10 rows):")
print(df.head(10).to_string(index=False))

# Save to CSV
csv_filename = 'merged_ibb_indices.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"\n💾 Saved to: {csv_filename}")

# Show statistics
print("\n" + "="*70)
print("📈 STATISTICS")
print("="*70)

print("\n🔢 Numeric columns summary:")
numeric_cols = ['INDEX_YASAM_KALITESI', 'INDEX_YURUNEBILIRLIK', 'KULTUREL_AKTIVITE_INDEX']
for col in numeric_cols:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"  Mean: {df[col].mean():.2f}")
        print(f"  Min: {df[col].min():.2f}")
        print(f"  Max: {df[col].max():.2f}")
        print(f"  Null count: {df[col].isna().sum()}")

print("\n📊 Records by İlçe:")
ilce_counts = df['ilce'].value_counts().sort_index()
for ilce, count in ilce_counts.items():
    print(f"  • {ilce}: {count} mahalles")

print("\n" + "="*70)
print("✅ EXPORT COMPLETE!")
print("="*70)

📄 EXPORTING MERGED DATA TO CSV

✅ Loaded: 407 locations

📊 DataFrame created:
  Rows: 407
  Columns: ['ilce', 'mahalle', 'INDEX_YASAM_KALITESI', 'INDEX_YURUNEBILIRLIK', 'KULTUREL_AKTIVITE_INDEX']

📋 Sample data (first 10 rows):
        ilce    mahalle  INDEX_YASAM_KALITESI  INDEX_YURUNEBILIRLIK  KULTUREL_AKTIVITE_INDEX
      BEYKOZ       RİVA             15.770645             38.211884                 0.000000
      BEYKOZ POLONEZKÖY             42.225281             40.374688                 1.125000
BÜYÜKÇEKMECE    GÜZELCE             49.512145             47.239770                 0.000000
KÜÇÜKÇEKMECE   İSTASYON             45.976300             50.494510                 0.841237
     ÇATALCA  DAĞYENİCE             19.711860             46.847600                 0.000000
       TUZLA    AYDINLI             36.882727             49.731765                 0.575163
  ARNAVUTKÖY    BOYALIK             22.032714             35.438889                 4.239316
     ÇATALCA   HALLAÇLI     

In [28]:
import sqlite3
import pandas as pd

print("="*70)
print("📊 AGGREGATING DATA BY MAHALLE/KÖY")
print("="*70)

# Connect to the database
conn = sqlite3.connect('ibb_secim.db')
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = [table[0] for table in cursor.fetchall()]

print(f"\n📁 Total Tables: {len(tables)}")

# Combine all data from all tables
all_data = []

for table in tables:
    print(f"\n📥 Reading: {table}")
    df = pd.read_sql(f"SELECT * FROM {table}", conn)
    print(f"   Rows: {len(df)}")
    all_data.append(df)

# Concatenate all dataframes
combined_df = pd.concat(all_data, ignore_index=True)
print(f"\n✅ Combined total rows: {len(combined_df)}")

# Show all column names
print(f"\n📋 All columns in dataset:")
for idx, col in enumerate(combined_df.columns, 1):
    print(f"  {idx}. {col}")

# Identify numeric columns to average (excluding İlçe Adı and Mahalle/Köy)
numeric_columns = combined_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\n🔢 Numeric columns to average ({len(numeric_columns)}):")
for col in numeric_columns:
    print(f"  • {col}")

# Group by İlçe Adı and Mahalle/Köy and calculate mean for numeric columns
print("\n🔄 Calculating averages by İlçe and Mahalle/Köy...")

aggregated_df = combined_df.groupby(['İlçe Adı', 'Mahalle/Köy'], as_index=False)[numeric_columns].mean()

print(f"\n✅ Aggregated to {len(aggregated_df)} unique İlçe-Mahalle combinations")

# Round numeric values to 2 decimal places for readability
aggregated_df[numeric_columns] = aggregated_df[numeric_columns].round(2)

# Drop unwanted columns
columns_to_drop = ['Sıra No', 'Sandık No', 'Kayıtlı Seçmen Sayısı', 
                   'Oy Kullanan Seçmen Sayısı', 'İtirazsız Geçerli Oy Sayısı', 
                   'İtirazlı Geçerli Oy Sayısı']

existing_cols_to_drop = [col for col in columns_to_drop if col in aggregated_df.columns]
if existing_cols_to_drop:
    aggregated_df = aggregated_df.drop(columns=existing_cols_to_drop)
    print(f"\n🗑️ Dropped columns: {', '.join(existing_cols_to_drop)}")
    print(f"📊 Remaining columns: {len(aggregated_df.columns)}")

# Clean Mahalle/Köy column - convert to lowercase and remove 'MAH.'
aggregated_df['Mahalle/Köy'] = aggregated_df['Mahalle/Köy'].str.lower()
aggregated_df['Mahalle/Köy'] = aggregated_df['Mahalle/Köy'].str.replace(r'\s*mah\.?\s*', ' ', regex=True)
aggregated_df['Mahalle/Köy'] = aggregated_df['Mahalle/Köy'].str.strip()  # Remove extra spaces
print(f"✂️ Cleaned 'Mahalle/Köy' column (lowercase + removed 'mah.')")

# Show sample results
print("\n" + "="*70)
print("📊 SAMPLE AGGREGATED DATA (first 10 rows):")
print("="*70)
print(aggregated_df.head(10).to_string(index=False))

# Show example of a specific mahalle aggregation
sample_mahalle = 'AHMEDİYE MAH.'
if sample_mahalle in combined_df['Mahalle/Köy'].values:
    print("\n" + "="*70)
    print(f"📍 EXAMPLE: {sample_mahalle}")
    print("="*70)
    
    # Original rows
    original_rows = combined_df[combined_df['Mahalle/Köy'] == sample_mahalle]
    print(f"\n🔍 Original rows found: {len(original_rows)}")
    print("\nOriginal data:")
    print(original_rows.to_string(index=False))
    
    # Aggregated result
    aggregated_row = aggregated_df[aggregated_df['Mahalle/Köy'] == sample_mahalle]
    print(f"\n📊 Aggregated result (averages):")
    print(aggregated_row.to_string(index=False))

# Save aggregated data to new table
print("\n" + "="*70)
print("💾 SAVING AGGREGATED DATA")
print("="*70)

aggregated_df.to_sql('aggregated_by_mahalle', conn, if_exists='replace', index=False)
print("✅ Saved to table: 'aggregated_by_mahalle'")

# Also save to CSV
csv_filename = 'aggregated_mahalle_data.csv'
aggregated_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
print(f"✅ Saved to CSV: {csv_filename}")

# Summary statistics
print("\n" + "="*70)
print("📈 SUMMARY STATISTICS")
print("="*70)

# Count by İlçe
ilce_counts = aggregated_df['İlçe Adı'].value_counts().sort_index()
print("\nMahalle count by İlçe:")
for ilce, count in ilce_counts.items():
    print(f"  • {ilce}: {count} mahalles")

# Show some key statistics
print("\n📊 Key metrics (averages across all mahalles):")
key_columns = [
    'Kayıtlı Seçmen Sayısı',
    'Oy Kullanan Seçmen Sayısı', 
    'Toplam Geçerli Oy',
    'Toplam Geçersiz Oy'
]

for col in key_columns:
    if col in aggregated_df.columns:
        avg_val = aggregated_df[col].mean()
        print(f"  • {col}: {avg_val:.2f}")

conn.close()

print("\n" + "="*70)
print("✅ AGGREGATION COMPLETE!")
print("="*70)

📊 AGGREGATING DATA BY MAHALLE/KÖY

📁 Total Tables: 9

📥 Reading: aggregated_by_mahalle
   Rows: 261

📥 Reading: bagcilar_ibb
   Rows: 1453

📥 Reading: bakırkoy_ibb
   Rows: 501

📥 Reading: besiktas_ibb
   Rows: 415

📥 Reading: beykoz_ibb
   Rows: 550

📥 Reading: esenyurt_ibb
   Rows: 1631

📥 Reading: fatih_ibb
   Rows: 838

📥 Reading: kadıkoy_ibb
   Rows: 1096

📥 Reading: uskudar_ibb
   Rows: 1190

✅ Combined total rows: 7935

📋 All columns in dataset:
  1. İlçe Adı
  2. Mahalle/Köy
  3. Sıra No
  4. Sandık No
  5. Kayıtlı Seçmen Sayısı
  6. Oy Kullanan Seçmen Sayısı
  7. İtirazsız Geçerli Oy Sayısı
  8. İtirazlı Geçerli Oy Sayısı
  9. Toplam Geçerli Oy
  10. Toplam Geçersiz Oy
  11. SAADET
  12. VATAN PARTİSİ
  13. CHP
  14. AK PARTİ
  15. İl Adı

🔢 Numeric columns to average (12):
  • Sıra No
  • Sandık No
  • Kayıtlı Seçmen Sayısı
  • Oy Kullanan Seçmen Sayısı
  • İtirazsız Geçerli Oy Sayısı
  • İtirazlı Geçerli Oy Sayısı
  • Toplam Geçerli Oy
  • Toplam Geçersiz Oy
  • SAADET
  • V

In [35]:
import json
import pandas as pd

print("="*70)
print("📄 EXPORTING MAHALLE PLACES DATA TO CSV")
print("="*70)

# Load the mahalle places JSON file
with open('mahalle_places_data_20251025_023357.json', 'r', encoding='utf-8') as f:
    places_data = json.load(f)

print(f"\n✅ Loaded: {places_data['metadata']['total_mahalles']} mahalles")
print(f"📊 Place types: {', '.join(places_data['metadata']['place_types'])}")

# Extract results and flatten the nested places structure
rows = []
for result in places_data['results']:
    row = {
        'mahalle': result['mahalle'],
        'district': result['district'],
        'centroid_lat': result['centroid_lat'],
        'centroid_lng': result['centroid_lng']
    }
    
    # Add each place type count as a separate column
    for place_type, count in result['places'].items():
        row[place_type] = count
    
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

# Clean mahalle column - remove 'Mahallesi' or 'Mahalle' suffix
df['mahalle'] = df['mahalle'].str.replace(r'\s+(Mahallesi|MAHALLESI|Mahalle|MAHALLE)$', '', regex=True, case=False)
df['mahalle'] = df['mahalle'].str.strip()  # Remove any trailing spaces

print(f"\n📊 DataFrame created:")
print(f"  Rows: {len(df)}")
print(f"  Columns: {list(df.columns)}")
print(f"✂️ Cleaned mahalle names (removed 'Mahallesi'/'Mahalle' suffixes)")

# Show sample data
print("\n📋 Sample data (first 5 rows):")
print(df.head(5).to_string(index=False))

# Save to CSV
csv_filename = 'mahalle_places_data.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"\n💾 Saved to: {csv_filename}")

# Show statistics
print("\n" + "="*70)
print("📈 STATISTICS")
print("="*70)

print("\n📊 Records by District:")
district_counts = df['district'].value_counts().sort_index()
for district, count in district_counts.items():
    print(f"  • {district}: {count} mahalles")

print("\n🔢 Top 5 place types by total count:")
place_columns = places_data['metadata']['place_types']
place_totals = {}
for col in place_columns:
    if col in df.columns:
        place_totals[col] = df[col].sum()

for place_type, total in sorted(place_totals.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"  • {place_type}: {int(total)} total")

print("\n" + "="*70)
print("✅ EXPORT COMPLETE!")
print("="*70)

📄 EXPORTING MAHALLE PLACES DATA TO CSV

✅ Loaded: 164 mahalles
📊 Place types: restaurant, library, school, park, atm, cafe, pharmacy, hospital, mosque, bus_station, train_station, transit_station

📊 DataFrame created:
  Rows: 164
  Columns: ['mahalle', 'district', 'centroid_lat', 'centroid_lng', 'restaurant', 'library', 'school', 'park', 'atm', 'cafe', 'pharmacy', 'hospital', 'mosque', 'bus_station', 'train_station', 'transit_station']
✂️ Cleaned mahalle names (removed 'Mahallesi'/'Mahalle' suffixes)

📋 Sample data (first 5 rows):
  mahalle district  centroid_lat  centroid_lng  restaurant  library  school  park  atm  cafe  pharmacy  hospital  mosque  bus_station  train_station  transit_station
 Balmumcu Beşiktaş     41.059527     29.015073           0        0      13     2    5     4         3         2       1            1              0                5
    Bebek Beşiktaş     41.078970     29.043979          11        1       4    12   13    16         6         1       4           

In [37]:
aggregated_df['Mahalle/Köy'] = aggregated_df['Mahalle/Köy'].str.replace(r'mah\.?', '', case=False, regex=True).str.strip().str.lower()

In [38]:
aggregated_df

,İlçe Adı,Mahalle/Köy,Toplam Geçerli Oy,Toplam Geçersiz Oy,SAADET,VATAN PARTİSİ,CHP,AK PARTİ
0,BAKIRKÖY,ataköy 1. kisim,278.75,4.50,0.25,0.75,234.75,43.00
1,BAKIRKÖY,ataköy 2-5-6. kisim,289.48,4.32,1.19,0.48,241.06,46.68
2,BAKIRKÖY,ataköy 3-4-11. kisim,293.53,3.32,0.58,0.63,255.84,36.37
3,BAKIRKÖY,ataköy 7-8-9-10. kisim,292.47,3.60,0.82,0.60,254.16,36.79
4,BAKIRKÖY,bakirköy kadin kapali ceza i̇nfaz kurumu,182.00,6.00,1.00,0.50,125.50,54.50
...,...,...,...,...,...,...,...,...
256,ÜSKÜDAR,yavuztürk,282.37,5.79,1.66,0.60,121.78,157.96
257,ÜSKÜDAR,zeynep kami̇l,275.70,4.43,1.63,0.43,167.57,105.83
258,ÜSKÜDAR,çengelköy,285.31,5.25,2.00,0.28,125.59,157.16
259,ÜSKÜDAR,ünalan,284.85,5.36,1.87,0.49,143.85,138.24


In [39]:
import pandas as pd
import re

print("="*70)
print("🔗 MERGING ALL THREE CSV FILES BY MAHALLE")
print("="*70)

# Load all three CSV files
print("\n📥 Loading CSV files...")
df_ibb = pd.read_csv('merged_ibb_indices.csv', encoding='utf-8-sig')
df_election = pd.read_csv('aggregated_mahalle_data.csv', encoding='utf-8-sig')
df_places = pd.read_csv('mahalle_places_data.csv', encoding='utf-8-sig')

print(f"✅ Loaded merged_ibb_indices.csv: {len(df_ibb)} rows")
print(f"✅ Loaded aggregated_mahalle_data.csv: {len(df_election)} rows")
print(f"✅ Loaded mahalle_places_data.csv: {len(df_places)} rows")

# Create a standardized mahalle name function
def standardize_mahalle(name):
    """Standardize mahalle names for matching"""
    if pd.isna(name):
        return ""
    
    name = str(name).strip()
    
    # Convert to lowercase
    name = name.lower()
    
    # Remove common suffixes and abbreviations
    name = re.sub(r'\s+(mahallesi|mahalle|mah\.?|mh\.?)\s*$', '', name, flags=re.IGNORECASE)
    
    # Remove extra whitespace
    name = re.sub(r'\s+', ' ', name).strip()
    
    # Remove dots
    name = name.replace('.', '')
    
    return name

# Create standardized columns for matching
print("\n🔄 Standardizing mahalle names...")

df_ibb['mahalle_std'] = df_ibb['mahalle'].apply(standardize_mahalle)
df_ibb['ilce_std'] = df_ibb['ilce'].str.upper().str.strip()

df_election['mahalle_std'] = df_election['Mahalle/Köy'].apply(standardize_mahalle)
df_election['ilce_std'] = df_election['İlçe Adı'].str.upper().str.strip()

df_places['mahalle_std'] = df_places['mahalle'].apply(standardize_mahalle)
df_places['ilce_std'] = df_places['district'].str.upper().str.strip()

print("✅ Standardization complete")

# Show sample standardized names
print("\n📋 Sample standardized names:")
print("\nIBB Indices:")
print(df_ibb[['ilce', 'mahalle', 'mahalle_std']].head(5).to_string(index=False))
print("\nElection Data:")
print(df_election[['İlçe Adı', 'Mahalle/Köy', 'mahalle_std']].head(5).to_string(index=False))
print("\nPlaces Data:")
print(df_places[['district', 'mahalle', 'mahalle_std']].head(5).to_string(index=False))

# Merge strategy: Start with places data (smallest), then add election data, then IBB indices
print("\n" + "="*70)
print("🔗 PERFORMING MERGES")
print("="*70)

# Step 1: Merge places with election data on mahalle_std and ilce_std
print("\n[1/2] Merging places data with election data...")
merged_step1 = pd.merge(
    df_places,
    df_election,
    left_on=['mahalle_std', 'ilce_std'],
    right_on=['mahalle_std', 'ilce_std'],
    how='left',
    suffixes=('', '_election')
)
print(f"✅ Result: {len(merged_step1)} rows")

# Step 2: Merge with IBB indices
print("\n[2/2] Merging with IBB indices...")
merged_final = pd.merge(
    merged_step1,
    df_ibb,
    left_on=['mahalle_std', 'ilce_std'],
    right_on=['mahalle_std', 'ilce_std'],
    how='left',
    suffixes=('', '_ibb')
)
print(f"✅ Result: {len(merged_final)} rows")

# Clean up the final dataframe - select and rename columns
print("\n🧹 Cleaning up merged data...")

# Select columns to keep
columns_to_keep = [
    # Location info
    'mahalle', 'district', 'centroid_lat', 'centroid_lng',
    
    # IBB Indices
    'INDEX_YASAM_KALITESI', 'INDEX_YURUNEBILIRLIK', 'KULTUREL_AKTIVITE_INDEX',
    
    # Place counts
    'restaurant', 'library', 'school', 'park', 'atm', 'cafe',
    'pharmacy', 'hospital', 'mosque', 'bus_station', 'train_station', 'transit_station',
    
    # Election data (selected columns)
    'Toplam Geçerli Oy', 'Toplam Geçersiz Oy',
    'CHP', 'AK PARTİ', 'SAADET', 'VATAN PARTİSİ'
]

# Keep only columns that exist
columns_to_keep = [col for col in columns_to_keep if col in merged_final.columns]
final_df = merged_final[columns_to_keep].copy()

# Rename for clarity
final_df = final_df.rename(columns={
    'mahalle': 'Mahalle',
    'district': 'İlçe',
    'centroid_lat': 'Enlem',
    'centroid_lng': 'Boylam'
})

print(f"✅ Final dataframe: {len(final_df)} rows, {len(final_df.columns)} columns")

# Show merge statistics
print("\n" + "="*70)
print("📊 MERGE STATISTICS")
print("="*70)

print(f"\n📍 Total neighborhoods: {len(final_df)}")
print(f"\n📊 Data completeness:")
print(f"  • With IBB Indices: {final_df['INDEX_YASAM_KALITESI'].notna().sum()} ({final_df['INDEX_YASAM_KALITESI'].notna().sum()/len(final_df)*100:.1f}%)")
print(f"  • With Election Data: {final_df['Toplam Geçerli Oy'].notna().sum()} ({final_df['Toplam Geçerli Oy'].notna().sum()/len(final_df)*100:.1f}%)")
print(f"  • With Places Data: {len(final_df)} (100.0%)")

print("\n🏘️ By district:")
district_counts = final_df['İlçe'].value_counts().sort_index()
for district, count in district_counts.items():
    print(f"  • {district}: {count} mahalles")

# Show sample merged data
print("\n" + "="*70)
print("📋 SAMPLE MERGED DATA (first 5 rows)")
print("="*70)
print(final_df.head(5).to_string(index=False))

# Save to CSV
output_filename = 'istanbul_mahalle_complete_data.csv'
final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print("\n" + "="*70)
print("💾 SAVED TO FILE")
print("="*70)
print(f"✅ Filename: {output_filename}")
print(f"📊 Total rows: {len(final_df)}")
print(f"📊 Total columns: {len(final_df.columns)}")
print(f"\n📋 Column list:")
for idx, col in enumerate(final_df.columns, 1):
    print(f"  {idx}. {col}")

print("\n" + "="*70)
print("✅ MERGE COMPLETE!")
print("="*70)

🔗 MERGING ALL THREE CSV FILES BY MAHALLE

📥 Loading CSV files...
✅ Loaded merged_ibb_indices.csv: 407 rows
✅ Loaded aggregated_mahalle_data.csv: 261 rows
✅ Loaded mahalle_places_data.csv: 164 rows

🔄 Standardizing mahalle names...
✅ Standardization complete

📋 Sample standardized names:

IBB Indices:
        ilce    mahalle mahalle_std
      BEYKOZ       RİVA       ri̇va
      BEYKOZ POLONEZKÖY  polonezköy
BÜYÜKÇEKMECE    GÜZELCE     güzelce
KÜÇÜKÇEKMECE   İSTASYON   i̇stasyon
     ÇATALCA  DAĞYENİCE  dağyeni̇ce

Election Data:
İlçe Adı                             Mahalle/Köy                              mahalle_std
BAKIRKÖY                    ATAKÖY 1. KISIM MAH.                           ataköy 1 kisim
BAKIRKÖY                ATAKÖY 2-5-6. KISIM MAH.                       ataköy 2-5-6 kisim
BAKIRKÖY               ATAKÖY 3-4-11. KISIM MAH.                      ataköy 3-4-11 kisim
BAKIRKÖY             ATAKÖY 7-8-9-10. KISIM MAH.                    ataköy 7-8-9-10 kisim
BAKIRKÖY BAKIRKÖ

In [40]:
import pandas as pd
import re
from difflib import SequenceMatcher

print("="*70)
print("🔍 ANALYZING MATCHING ISSUES")
print("="*70)

# Load the datasets
df_places = pd.read_csv('mahalle_places_data.csv', encoding='utf-8-sig')
df_election = pd.read_csv('aggregated_mahalle_data.csv', encoding='utf-8-sig')
df_ibb = pd.read_csv('merged_ibb_indices.csv', encoding='utf-8-sig')

print(f"\n📊 Dataset sizes:")
print(f"  Places: {len(df_places)} rows")
print(f"  Election: {len(df_election)} rows")
print(f"  IBB: {len(df_ibb)} rows")

# Show sample mahalle names from each dataset for comparison
print("\n" + "="*70)
print("📋 SAMPLE MAHALLE NAMES BY DATASET")
print("="*70)

print("\n🔹 Places Data (sample from Beşiktaş):")
besiktas_places = df_places[df_places['district'].str.upper() == 'BEŞIKTAŞ']['mahalle'].head(10)
for name in besiktas_places:
    print(f"  • {name}")

print("\n🔹 Election Data (sample from BEŞİKTAŞ):")
besiktas_election = df_election[df_election['İlçe Adı'].str.upper() == 'BEŞİKTAŞ']['Mahalle/Köy'].head(10)
for name in besiktas_election:
    print(f"  • {name}")

print("\n🔹 IBB Data (sample from BEŞIKTAŞ):")
besiktas_ibb = df_ibb[df_ibb['ilce'].str.upper() == 'BEŞIKTAŞ']['mahalle'].head(10)
for name in besiktas_ibb:
    print(f"  • {name}")

# Check for specific mismatches
print("\n" + "="*70)
print("🔍 CHECKING SPECIFIC DISTRICT MATCHING")
print("="*70)

# For each district in places data, check election data coverage
districts_in_places = df_places['district'].unique()
print(f"\n📊 Districts in places data: {len(districts_in_places)}")

for district in sorted(districts_in_places):
    places_count = len(df_places[df_places['district'] == district])
    
    # Try to find matching district in election data (case-insensitive)
    election_matches = df_election[df_election['İlçe Adı'].str.upper() == district.upper()]
    election_count = len(election_matches)
    
    print(f"\n{district}:")
    print(f"  Places data: {places_count} mahalles")
    print(f"  Election data: {election_count} mahalles")
    
    if election_count > 0:
        # Show sample names from both
        places_names = set(df_places[df_places['district'] == district]['mahalle'].str.lower())
        election_names = set(election_matches['Mahalle/Köy'].str.lower())
        
        # Check for differences
        places_only = places_names - election_names
        election_only = election_names - places_names
        matched = places_names & election_names
        
        print(f"  Matched: {len(matched)} mahalles")
        
        if len(places_only) > 0:
            print(f"  In places but not election: {len(places_only)}")
            for name in sorted(list(places_only)[:3]):
                print(f"    - {name}")
        
        if len(election_only) > 0:
            print(f"  In election but not places: {len(election_only)}")
            for name in sorted(list(election_only)[:3]):
                print(f"    - {name}")

🔍 ANALYZING MATCHING ISSUES

📊 Dataset sizes:
  Places: 164 rows
  Election: 261 rows
  IBB: 407 rows

📋 SAMPLE MAHALLE NAMES BY DATASET

🔹 Places Data (sample from Beşiktaş):
  • Balmumcu
  • Bebek
  • Kültür
  • Kuruçeşme
  • Mecidiye
  • Nispetiye
  • Yıldız
  • Türkali
  • Cihannüma
  • Vişnezade

🔹 Election Data (sample from BEŞİKTAŞ):
  • ABBASAĞA MAH.
  • AKAT MAH.
  • ARNAVUTKÖY MAH.
  • BALMUMCU MAH.
  • BEBEK MAH.
  • CİHANNÜMA MAH.
  • DİKİLİTAŞ MAH.
  • ETİLER MAH.
  • GAYRETTEPE MAH.
  • KONAKLAR MAH.

🔹 IBB Data (sample from BEŞIKTAŞ):

🔍 CHECKING SPECIFIC DISTRICT MATCHING

📊 Districts in places data: 8

Bakırköy:
  Places data: 15 mahalles
  Election data: 16 mahalles
  Matched: 0 mahalles
  In places but not election: 15
    - ataköy 3-4-11. kısım
    - kartaltepe
    - yeşilköy
  In election but not places: 16
    - yeni̇mahalle mah.
    - zeyti̇nli̇k mah.
    - şenli̇kköy mah.

Bağcılar:
  Places data: 15 mahalles
  Election data: 22 mahalles
  Matched: 0 mahalles
  

In [41]:
import pandas as pd
import re
from difflib import get_close_matches
import unicodedata

print("="*70)
print("🔗 IMPROVED MERGE WITH FUZZY MATCHING")
print("="*70)

# Load all three CSV files
df_places = pd.read_csv('mahalle_places_data.csv', encoding='utf-8-sig')
df_election = pd.read_csv('aggregated_mahalle_data.csv', encoding='utf-8-sig')
df_ibb = pd.read_csv('merged_ibb_indices.csv', encoding='utf-8-sig')

print(f"\n📥 Loaded datasets:")
print(f"  Places: {len(df_places)} rows")
print(f"  Election: {len(df_election)} rows")
print(f"  IBB: {len(df_ibb)} rows")

def normalize_text(text):
    """Aggressive normalization for better matching"""
    if pd.isna(text):
        return ""
    
    text = str(text).strip()
    
    # Normalize Unicode characters (İ, ı, etc.)
    text = unicodedata.normalize('NFKD', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove common suffixes
    text = re.sub(r'\s+(mahallesi|mahalle|mah\.?|mh\.?)\s*$', '', text, flags=re.IGNORECASE)
    
    # Remove dots and extra spaces
    text = text.replace('.', '').replace('-', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Handle Turkish characters consistently
    replacements = {
        'ı': 'i', 'ğ': 'g', 'ü': 'u', 'ş': 's', 'ö': 'o', 'ç': 'c',
        'İ': 'i', 'Ğ': 'g', 'Ü': 'u', 'Ş': 's', 'Ö': 'o', 'Ç': 'c'
    }
    for tr_char, en_char in replacements.items():
        text = text.replace(tr_char, en_char)
    
    return text

# Create normalized columns
print("\n🔄 Normalizing mahalle names...")

df_places['mahalle_norm'] = df_places['mahalle'].apply(normalize_text)
df_places['district_norm'] = df_places['district'].apply(normalize_text)

df_election['mahalle_norm'] = df_election['Mahalle/Köy'].apply(normalize_text)
df_election['district_norm'] = df_election['İlçe Adı'].apply(normalize_text)

df_ibb['mahalle_norm'] = df_ibb['mahalle'].apply(normalize_text)
df_ibb['district_norm'] = df_ibb['ilce'].apply(normalize_text)

print("✅ Normalization complete")

# Create a manual merge with fuzzy matching
print("\n" + "="*70)
print("🔗 PERFORMING FUZZY MERGE")
print("="*70)

# Start with places data and add columns
result_df = df_places.copy()

# Initialize election columns
election_cols = ['Toplam Geçerli Oy', 'Toplam Geçersiz Oy', 'CHP', 'AK PARTİ', 'SAADET', 'VATAN PARTİSİ']
for col in election_cols:
    result_df[col] = None

# Initialize IBB columns
ibb_cols = ['INDEX_YASAM_KALITESI', 'INDEX_YURUNEBILIRLIK', 'KULTUREL_AKTIVITE_INDEX']
for col in ibb_cols:
    result_df[col] = None

matched_election = 0
matched_ibb = 0

print("\n🔄 Matching each mahalle...")

for idx, row in result_df.iterrows():
    mahalle_norm = row['mahalle_norm']
    district_norm = row['district_norm']
    
    # Try exact match first for election data
    election_match = df_election[
        (df_election['mahalle_norm'] == mahalle_norm) & 
        (df_election['district_norm'] == district_norm)
    ]
    
    # If no exact match, try fuzzy matching within the same district
    if len(election_match) == 0:
        district_elections = df_election[df_election['district_norm'] == district_norm]
        if len(district_elections) > 0:
            candidates = district_elections['mahalle_norm'].tolist()
            matches = get_close_matches(mahalle_norm, candidates, n=1, cutoff=0.8)
            if matches:
                election_match = district_elections[district_elections['mahalle_norm'] == matches[0]]
    
    if len(election_match) > 0:
        for col in election_cols:
            result_df.at[idx, col] = election_match.iloc[0][col]
        matched_election += 1
    
    # Try exact match first for IBB data
    ibb_match = df_ibb[
        (df_ibb['mahalle_norm'] == mahalle_norm) & 
        (df_ibb['district_norm'] == district_norm)
    ]
    
    # If no exact match, try fuzzy matching within the same district
    if len(ibb_match) == 0:
        district_ibb = df_ibb[df_ibb['district_norm'] == district_norm]
        if len(district_ibb) > 0:
            candidates = district_ibb['mahalle_norm'].tolist()
            matches = get_close_matches(mahalle_norm, candidates, n=1, cutoff=0.8)
            if matches:
                ibb_match = district_ibb[district_ibb['mahalle_norm'] == matches[0]]
    
    if len(ibb_match) > 0:
        for col in ibb_cols:
            result_df.at[idx, col] = ibb_match.iloc[0][col]
        matched_ibb += 1

print(f"\n✅ Matching complete:")
print(f"  Election data matched: {matched_election}/{len(result_df)} ({matched_election/len(result_df)*100:.1f}%)")
print(f"  IBB data matched: {matched_ibb}/{len(result_df)} ({matched_ibb/len(result_df)*100:.1f}%)")

# Clean up final dataframe
final_df = result_df[[
    'mahalle', 'district', 'centroid_lat', 'centroid_lng',
    'INDEX_YASAM_KALITESI', 'INDEX_YURUNEBILIRLIK', 'KULTUREL_AKTIVITE_INDEX',
    'restaurant', 'library', 'school', 'park', 'atm', 'cafe',
    'pharmacy', 'hospital', 'mosque', 'bus_station', 'train_station', 'transit_station',
    'Toplam Geçerli Oy', 'Toplam Geçersiz Oy', 'CHP', 'AK PARTİ', 'SAADET', 'VATAN PARTİSİ'
]].copy()

# Rename columns
final_df = final_df.rename(columns={
    'mahalle': 'Mahalle',
    'district': 'İlçe',
    'centroid_lat': 'Enlem',
    'centroid_lng': 'Boylam'
})

# Show statistics by district
print("\n" + "="*70)
print("📊 STATISTICS BY DISTRICT")
print("="*70)

for district in sorted(final_df['İlçe'].unique()):
    district_df = final_df[final_df['İlçe'] == district]
    total = len(district_df)
    with_election = district_df['Toplam Geçerli Oy'].notna().sum()
    with_ibb = district_df['INDEX_YASAM_KALITESI'].notna().sum()
    
    print(f"\n{district}: {total} mahalles")
    print(f"  Election data: {with_election}/{total} ({with_election/total*100:.0f}%)")
    print(f"  IBB indices: {with_ibb}/{total} ({with_ibb/total*100:.0f}%)")

# Show sample merged data
print("\n" + "="*70)
print("📋 SAMPLE MERGED DATA (with election data)")
print("="*70)
sample_with_data = final_df[final_df['Toplam Geçerli Oy'].notna()].head(5)
print(sample_with_data[['Mahalle', 'İlçe', 'Toplam Geçerli Oy', 'CHP', 'AK PARTİ', 'restaurant', 'INDEX_YASAM_KALITESI']].to_string(index=False))

# Save to CSV
output_filename = 'istanbul_mahalle_complete_data.csv'
final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print("\n" + "="*70)
print("💾 SAVED TO FILE")
print("="*70)
print(f"✅ Filename: {output_filename}")
print(f"📊 Total rows: {len(final_df)}")
print(f"📊 Rows with election data: {final_df['Toplam Geçerli Oy'].notna().sum()}")
print(f"📊 Rows with IBB indices: {final_df['INDEX_YASAM_KALITESI'].notna().sum()}")

print("\n" + "="*70)
print("✅ IMPROVED MERGE COMPLETE!")
print("="*70)

🔗 IMPROVED MERGE WITH FUZZY MATCHING

📥 Loaded datasets:
  Places: 164 rows
  Election: 261 rows
  IBB: 407 rows

🔄 Normalizing mahalle names...
✅ Normalization complete

🔗 PERFORMING FUZZY MERGE

🔄 Matching each mahalle...

✅ Matching complete:
  Election data matched: 92/164 (56.1%)
  IBB data matched: 27/164 (16.5%)

📊 STATISTICS BY DISTRICT

Bakırköy: 15 mahalles
  Election data: 14/15 (93%)
  IBB indices: 6/15 (40%)

Bağcılar: 15 mahalles
  Election data: 14/15 (93%)
  IBB indices: 1/15 (7%)

Beykoz: 16 mahalles
  Election data: 15/16 (94%)
  IBB indices: 8/16 (50%)

Beşiktaş: 11 mahalles
  Election data: 0/11 (0%)
  IBB indices: 0/11 (0%)

Esenyurt: 5 mahalles
  Election data: 4/5 (80%)
  IBB indices: 1/5 (20%)

Fatih: 57 mahalles
  Election data: 0/57 (0%)
  IBB indices: 0/57 (0%)

Kadıköy: 12 mahalles
  Election data: 12/12 (100%)
  IBB indices: 3/12 (25%)

Üsküdar: 33 mahalles
  Election data: 33/33 (100%)
  IBB indices: 8/33 (24%)

📋 SAMPLE MERGED DATA (with election data)
  

In [2]:
import pandas as pd
import re
import unicodedata

print("="*70)
print("📊 ADDING POPULATION DATA FROM EXCEL")
print("="*70)

# Load the Excel file
print("\n📥 Loading Excel file...")
df_excel = pd.read_excel('m2 başına mahalle yoğunluğu.xlsx')

print(f"✅ Loaded: {len(df_excel)} rows, {len(df_excel.columns)} columns")
print(f"\n📋 Column names:")
for idx, col in enumerate(df_excel.columns, 1):
    print(f"  {idx}. {col}")

# Show sample data
print("\n📄 Sample data (first 10 rows):")
print(df_excel.head(10).to_string(index=False))

# Load the current merged data
print("\n" + "="*70)
print("📥 Loading current istanbul_mahalle_complete_data.csv...")
df_main = pd.read_csv('istanbul_mahalle_complete_data.csv', encoding='utf-8-sig')

print(f"✅ Loaded: {len(df_main)} rows, {len(df_main.columns)} columns")

# Normalization function (same as before)
def normalize_text(text):
    """Aggressive normalization for better matching"""
    if pd.isna(text):
        return ""
    
    text = str(text).strip()
    text = unicodedata.normalize('NFKD', text)
    text = text.lower()
    text = re.sub(r'\s+(mahallesi|mahalle|mah\.?|mh\.?)\s*$', '', text, flags=re.IGNORECASE)
    text = text.replace('.', '').replace('-', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Handle Turkish characters
    replacements = {
        'ı': 'i', 'ğ': 'g', 'ü': 'u', 'ş': 's', 'ö': 'o', 'ç': 'c',
        'İ': 'i', 'Ğ': 'g', 'Ü': 'u', 'Ş': 's', 'Ö': 'o', 'Ç': 'c'
    }
    for tr_char, en_char in replacements.items():
        text = text.replace(tr_char, en_char)
    
    return text

# Identify mahalle and nüfus columns in Excel file
print("\n" + "="*70)
print("🔍 IDENTIFYING COLUMNS")
print("="*70)

# Try to find mahalle column (common variations)
mahalle_col = None
for col in df_excel.columns:
    if any(x in col.lower() for x in ['mahalle', 'neighborhood']):
        mahalle_col = col
        break

# Try to find nüfus column
nufus_col = None
for col in df_excel.columns:
    if any(x in col.lower() for x in ['nüfus', 'nufus', 'population', 'popülasyon']):
        nufus_col = col
        break

# Try to find ilçe/district column
ilce_col = None
for col in df_excel.columns:
    if any(x in col.lower() for x in ['ilçe', 'ilce', 'district']):
        ilce_col = col
        break

print(f"✅ Mahalle column: {mahalle_col}")
print(f"✅ Nüfus column: {nufus_col}")
print(f"✅ İlçe column: {ilce_col}")

if not mahalle_col or not nufus_col:
    print("\n❌ Could not identify mahalle or nüfus columns!")
    print("Please check the Excel file structure.")
else:
    # Normalize names in both datasets
    print("\n🔄 Normalizing names for matching...")
    
    df_excel['mahalle_norm'] = df_excel[mahalle_col].apply(normalize_text)
    df_main['mahalle_norm'] = df_main['Mahalle'].apply(normalize_text)
    
    if ilce_col:
        df_excel['ilce_norm'] = df_excel[ilce_col].apply(normalize_text)
        df_main['ilce_norm'] = df_main['İlçe'].apply(normalize_text)
    
    print("✅ Normalization complete")
    
    # Perform merge
    print("\n" + "="*70)
    print("🔗 MERGING POPULATION DATA")
    print("="*70)
    
    # Create a mapping from normalized names to nüfus values
    nufus_mapping = {}
    
    for idx, row in df_excel.iterrows():
        mahalle_norm = row['mahalle_norm']
        ilce_norm = row['ilce_norm'] if ilce_col and 'ilce_norm' in row else None
        nufus = row[nufus_col]
        
        if ilce_norm:
            key = f"{ilce_norm}_{mahalle_norm}"
        else:
            key = mahalle_norm
        
        nufus_mapping[key] = nufus
    
    # Add nüfus column to main dataframe
    df_main['Nüfus'] = None
    matched = 0
    
    for idx, row in df_main.iterrows():
        mahalle_norm = row['mahalle_norm']
        ilce_norm = row['ilce_norm'] if 'ilce_norm' in row else None
        
        # Try matching with district first
        if ilce_norm:
            key = f"{ilce_norm}_{mahalle_norm}"
            if key in nufus_mapping:
                df_main.at[idx, 'Nüfus'] = nufus_mapping[key]
                matched += 1
                continue
        
        # Try matching without district
        if mahalle_norm in nufus_mapping:
            df_main.at[idx, 'Nüfus'] = nufus_mapping[mahalle_norm]
            matched += 1
    
    print(f"\n✅ Matched {matched}/{len(df_main)} rows ({matched/len(df_main)*100:.1f}%)")
    
    # Convert Nüfus to numeric type
    df_main['Nüfus'] = pd.to_numeric(df_main['Nüfus'], errors='coerce')
    
    # Clean up temporary columns
    df_main = df_main.drop(columns=['mahalle_norm', 'ilce_norm'], errors='ignore')
    
    # Show statistics
    print("\n" + "="*70)
    print("📊 POPULATION STATISTICS")
    print("="*70)
    
    if df_main['Nüfus'].notna().sum() > 0:
        print(f"\nRows with population data: {df_main['Nüfus'].notna().sum()}/{len(df_main)}")
        print(f"\n📈 Population stats:")
        print(f"  Total population: {df_main['Nüfus'].sum():,.0f}")
        print(f"  Average: {df_main['Nüfus'].mean():,.0f}")
        print(f"  Median: {df_main['Nüfus'].median():,.0f}")
        print(f"  Min: {df_main['Nüfus'].min():,.0f}")
        print(f"  Max: {df_main['Nüfus'].max():,.0f}")
        
        print("\n🏘️ Top 10 most populated neighborhoods:")
        top10 = df_main[df_main['Nüfus'].notna()].nlargest(10, 'Nüfus')[['Mahalle', 'İlçe', 'Nüfus']]
        print(top10.to_string(index=False))
    
    # Show sample merged data
    print("\n" + "="*70)
    print("📋 SAMPLE UPDATED DATA")
    print("="*70)
    sample_cols = ['Mahalle', 'İlçe', 'Nüfus', 'restaurant', 'school', 'Toplam Geçerli Oy']
    available_cols = [col for col in sample_cols if col in df_main.columns]
    print(df_main[available_cols].head(10).to_string(index=False))
    
    # Save updated data
    output_filename = 'istanbul_mahalle_complete_data.csv'
    df_main.to_csv(output_filename, index=False, encoding='utf-8-sig')
    
    print("\n" + "="*70)
    print("💾 SAVED UPDATED FILE")
    print("="*70)
    print(f"✅ Filename: {output_filename}")
    print(f"📊 Total rows: {len(df_main)}")
    print(f"📊 Total columns: {len(df_main.columns)}")
    print(f"📊 Rows with population: {df_main['Nüfus'].notna().sum()}")
    
    print("\n📋 Updated column list:")
    for idx, col in enumerate(df_main.columns, 1):
        print(f"  {idx}. {col}")
    
    print("\n" + "="*70)
    print("✅ POPULATION DATA ADDED SUCCESSFULLY!")
    print("="*70)

📊 ADDING POPULATION DATA FROM EXCEL

📥 Loading Excel file...
✅ Loaded: 954 rows, 3 columns

📋 Column names:
  1. Mahalle
  2. İlçe
  3. Nüfus

📄 Sample data (first 10 rows):
             Mahalle         İlçe   Nüfus
           Yeşiltepe  Zeytinburnu 96576.0
               Fatih      Esenler 89562.0
Mareşal Fevzi Çakmak     Güngören 86603.0
           Nenehatun      Esenler 86504.0
             Yenigün     Bağcılar 84843.0
     Kâzım Karabekir      Esenler 84393.0
            Hürriyet Bahçelievler 82759.0
           Akıncılar     Güngören 82208.0
         Siyavuşpaşa Bahçelievler 81140.0
           Davutpaşa      Esenler 79991.0

📥 Loading current istanbul_mahalle_complete_data.csv...
✅ Loaded: 164 rows, 25 columns

🔍 IDENTIFYING COLUMNS
✅ Mahalle column: Mahalle
✅ Nüfus column: Nüfus
✅ İlçe column: None

🔄 Normalizing names for matching...
✅ Normalization complete

🔗 MERGING POPULATION DATA

✅ Matched 134/164 rows (81.7%)

📊 POPULATION STATISTICS

Rows with population data: 134/164

📈 

In [5]:
import pandas as pd
import re
import unicodedata

print("="*70)
print("📊 ADDING BUILDING & EARTHQUAKE DATA TO DATASET")
print("="*70)

# Load the two new CSV files
print("\n📥 Loading new CSV files...")

# Try different encodings for Turkish text
try:
    df_buildings = pd.read_csv('2017-yl-mahalle-bazl-bina-saylar.csv', encoding='utf-8-sig', sep=';')
except UnicodeDecodeError:
    try:
        df_buildings = pd.read_csv('2017-yl-mahalle-bazl-bina-saylar.csv', encoding='cp1254', sep=';')
    except UnicodeDecodeError:
        df_buildings = pd.read_csv('2017-yl-mahalle-bazl-bina-saylar.csv', encoding='latin1', sep=';')

try:
    df_earthquake = pd.read_csv('deprem-senaryosu-analiz-sonuclar.csv', encoding='utf-8-sig', sep=';')
except UnicodeDecodeError:
    try:
        df_earthquake = pd.read_csv('deprem-senaryosu-analiz-sonuclar.csv', encoding='cp1254', sep=';')
    except UnicodeDecodeError:
        df_earthquake = pd.read_csv('deprem-senaryosu-analiz-sonuclar.csv', encoding='latin1', sep=';')

print(f"✅ Loaded buildings data: {len(df_buildings)} rows, {len(df_buildings.columns)} columns")
print(f"✅ Loaded earthquake data: {len(df_earthquake)} rows, {len(df_earthquake.columns)} columns")

# Show column names
print("\n📋 Buildings CSV columns:")
for idx, col in enumerate(df_buildings.columns, 1):
    print(f"  {idx}. {col}")

print("\n📋 Earthquake CSV columns:")
for idx, col in enumerate(df_earthquake.columns, 1):
    print(f"  {idx}. {col}")

# Show sample data
print("\n📄 Sample buildings data (first 5 rows):")
print(df_buildings.head(5).to_string(index=False))

print("\n📄 Sample earthquake data (first 5 rows):")
print(df_earthquake.head(5).to_string(index=False))

# Load the current main dataset
print("\n" + "="*70)
print("📥 Loading current istanbul_mahalle_complete_data.csv...")
df_main = pd.read_csv('istanbul_mahalle_complete_data.csv', encoding='utf-8-sig')

print(f"✅ Loaded: {len(df_main)} rows, {len(df_main.columns)} columns")

# Normalization function (same as before)
def normalize_text(text):
    """Aggressive normalization for better matching"""
    if pd.isna(text):
        return ""
    
    text = str(text).strip()
    text = unicodedata.normalize('NFKD', text)
    text = text.lower()
    text = re.sub(r'\s+(mahallesi|mahalle|mah\.?|mh\.?)\s*$', '', text, flags=re.IGNORECASE)
    text = text.replace('.', '').replace('-', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Handle Turkish characters
    replacements = {
        'ı': 'i', 'ğ': 'g', 'ü': 'u', 'ş': 's', 'ö': 'o', 'ç': 'c',
        'İ': 'i', 'Ğ': 'g', 'Ü': 'u', 'Ş': 's', 'Ö': 'o', 'Ç': 'c'
    }
    for tr_char, en_char in replacements.items():
        text = text.replace(tr_char, en_char)
    
    return text

# Identify columns in each CSV
print("\n" + "="*70)
print("🔍 IDENTIFYING COLUMNS IN NEW FILES")
print("="*70)

# Buildings CSV
buildings_mahalle_col = None
buildings_ilce_col = None
for col in df_buildings.columns:
    if any(x in col.lower() for x in ['mahalle', 'neighborhood']) and not buildings_mahalle_col:
        buildings_mahalle_col = col
    if any(x in col.lower() for x in ['ilçe', 'ilce', 'district']) and not buildings_ilce_col:
        buildings_ilce_col = col

print(f"\n🏗️ Buildings CSV:")
print(f"  Mahalle column: {buildings_mahalle_col}")
print(f"  İlçe column: {buildings_ilce_col}")

# Earthquake CSV
earthquake_mahalle_col = None
earthquake_ilce_col = None
for col in df_earthquake.columns:
    if any(x in col.lower() for x in ['mahalle', 'neighborhood']) and not earthquake_mahalle_col:
        earthquake_mahalle_col = col
    if any(x in col.lower() for x in ['ilçe', 'ilce', 'district']) and not earthquake_ilce_col:
        earthquake_ilce_col = col

print(f"\n🏚️ Earthquake CSV:")
print(f"  Mahalle column: {earthquake_mahalle_col}")
print(f"  İlçe column: {earthquake_ilce_col}")

# Normalize names in all datasets
print("\n" + "="*70)
print("🔄 NORMALIZING NAMES FOR MATCHING")
print("="*70)

# Main dataset
df_main['mahalle_norm'] = df_main['Mahalle'].apply(normalize_text)
df_main['ilce_norm'] = df_main['İlçe'].apply(normalize_text)

# Buildings dataset
if buildings_mahalle_col:
    df_buildings['mahalle_norm'] = df_buildings[buildings_mahalle_col].apply(normalize_text)
if buildings_ilce_col:
    df_buildings['ilce_norm'] = df_buildings[buildings_ilce_col].apply(normalize_text)

# Earthquake dataset
if earthquake_mahalle_col:
    df_earthquake['mahalle_norm'] = df_earthquake[earthquake_mahalle_col].apply(normalize_text)
if earthquake_ilce_col:
    df_earthquake['ilce_norm'] = df_earthquake[earthquake_ilce_col].apply(normalize_text)

print("✅ Normalization complete")

# Merge buildings data
print("\n" + "="*70)
print("🔗 MERGING BUILDINGS DATA")
print("="*70)

if buildings_mahalle_col:
    # Create mapping for buildings data
    buildings_data = {}
    
    # Get numeric columns to add (exclude mahalle and ilce columns)
    buildings_cols_to_add = [col for col in df_buildings.columns 
                            if col not in [buildings_mahalle_col, buildings_ilce_col, 'mahalle_norm', 'ilce_norm']
                            and df_buildings[col].dtype in ['int64', 'float64', 'Int64', 'Float64']]
    
    print(f"📊 Columns to add from buildings data: {len(buildings_cols_to_add)}")
    for col in buildings_cols_to_add[:10]:  # Show first 10
        print(f"  • {col}")
    if len(buildings_cols_to_add) > 10:
        print(f"  ... and {len(buildings_cols_to_add) - 10} more")
    
    # Build mapping
    for idx, row in df_buildings.iterrows():
        mahalle_norm = row['mahalle_norm']
        ilce_norm = row.get('ilce_norm', None)
        
        if ilce_norm:
            key = f"{ilce_norm}_{mahalle_norm}"
        else:
            key = mahalle_norm
        
        buildings_data[key] = {col: row[col] for col in buildings_cols_to_add}
    
    # Add columns to main dataframe
    for col in buildings_cols_to_add:
        df_main[col] = None
    
    matched_buildings = 0
    for idx, row in df_main.iterrows():
        mahalle_norm = row['mahalle_norm']
        ilce_norm = row['ilce_norm']
        
        # Try with district
        key = f"{ilce_norm}_{mahalle_norm}"
        if key in buildings_data:
            for col in buildings_cols_to_add:
                df_main.at[idx, col] = buildings_data[key].get(col)
            matched_buildings += 1
        # Try without district
        elif mahalle_norm in buildings_data:
            for col in buildings_cols_to_add:
                df_main.at[idx, col] = buildings_data[mahalle_norm].get(col)
            matched_buildings += 1
    
    print(f"\n✅ Matched buildings data: {matched_buildings}/{len(df_main)} rows ({matched_buildings/len(df_main)*100:.1f}%)")

# Merge earthquake data
print("\n" + "="*70)
print("🔗 MERGING EARTHQUAKE DATA")
print("="*70)

if earthquake_mahalle_col:
    # Create mapping for earthquake data
    earthquake_data = {}
    
    # Get numeric columns to add (exclude mahalle and ilce columns)
    earthquake_cols_to_add = [col for col in df_earthquake.columns 
                             if col not in [earthquake_mahalle_col, earthquake_ilce_col, 'mahalle_norm', 'ilce_norm']
                             and df_earthquake[col].dtype in ['int64', 'float64', 'Int64', 'Float64']]
    
    print(f"📊 Columns to add from earthquake data: {len(earthquake_cols_to_add)}")
    for col in earthquake_cols_to_add[:10]:  # Show first 10
        print(f"  • {col}")
    if len(earthquake_cols_to_add) > 10:
        print(f"  ... and {len(earthquake_cols_to_add) - 10} more")
    
    # Build mapping
    for idx, row in df_earthquake.iterrows():
        mahalle_norm = row['mahalle_norm']
        ilce_norm = row.get('ilce_norm', None)
        
        if ilce_norm:
            key = f"{ilce_norm}_{mahalle_norm}"
        else:
            key = mahalle_norm
        
        earthquake_data[key] = {col: row[col] for col in earthquake_cols_to_add}
    
    # Add columns to main dataframe
    for col in earthquake_cols_to_add:
        df_main[col] = None
    
    matched_earthquake = 0
    for idx, row in df_main.iterrows():
        mahalle_norm = row['mahalle_norm']
        ilce_norm = row['ilce_norm']
        
        # Try with district
        key = f"{ilce_norm}_{mahalle_norm}"
        if key in earthquake_data:
            for col in earthquake_cols_to_add:
                df_main.at[idx, col] = earthquake_data[key].get(col)
            matched_earthquake += 1
        # Try without district
        elif mahalle_norm in earthquake_data:
            for col in earthquake_cols_to_add:
                df_main.at[idx, col] = earthquake_data[mahalle_norm].get(col)
            matched_earthquake += 1
    
    print(f"\n✅ Matched earthquake data: {matched_earthquake}/{len(df_main)} rows ({matched_earthquake/len(df_main)*100:.1f}%)")

# Clean up temporary columns
df_main = df_main.drop(columns=['mahalle_norm', 'ilce_norm'], errors='ignore')

# Show summary statistics
print("\n" + "="*70)
print("📊 UPDATED DATASET SUMMARY")
print("="*70)

print(f"\nTotal rows: {len(df_main)}")
print(f"Total columns: {len(df_main.columns)} (was 26, added {len(df_main.columns) - 26} new columns)")

print("\n📋 Data completeness by source:")
print(f"  • Places data: 164/164 (100.0%)")
print(f"  • Population: {df_main['Nüfus'].notna().sum()}/164 ({df_main['Nüfus'].notna().sum()/164*100:.1f}%)")
print(f"  • Election data: {df_main['Toplam Geçerli Oy'].notna().sum()}/164 ({df_main['Toplam Geçerli Oy'].notna().sum()/164*100:.1f}%)")
print(f"  • IBB indices: {df_main['INDEX_YASAM_KALITESI'].notna().sum()}/164 ({df_main['INDEX_YASAM_KALITESI'].notna().sum()/164*100:.1f}%)")

if buildings_mahalle_col and buildings_cols_to_add:
    sample_col = buildings_cols_to_add[0]
    print(f"  • Buildings data: {df_main[sample_col].notna().sum()}/164 ({df_main[sample_col].notna().sum()/164*100:.1f}%)")

if earthquake_mahalle_col and earthquake_cols_to_add:
    sample_col = earthquake_cols_to_add[0]
    print(f"  • Earthquake data: {df_main[sample_col].notna().sum()}/164 ({df_main[sample_col].notna().sum()/164*100:.1f}%)")

# Show sample of new columns
print("\n" + "="*70)
print("📋 SAMPLE DATA WITH NEW FEATURES")
print("="*70)

sample_cols = ['Mahalle', 'İlçe', 'Nüfus']
if buildings_cols_to_add:
    sample_cols.extend(buildings_cols_to_add[:3])
if earthquake_cols_to_add:
    sample_cols.extend(earthquake_cols_to_add[:3])

available_cols = [col for col in sample_cols if col in df_main.columns]
print(df_main[available_cols].head(10).to_string(index=False))

# Save updated dataset
output_filename = 'istanbul_mahalle_complete_data.csv'
df_main.to_csv(output_filename, index=False, encoding='utf-8-sig')

print("\n" + "="*70)
print("💾 SAVED UPDATED FILE")
print("="*70)
print(f"✅ Filename: {output_filename}")
print(f"📊 Total rows: {len(df_main)}")
print(f"📊 Total columns: {len(df_main.columns)}")

print("\n📋 All columns in dataset:")
for idx, col in enumerate(df_main.columns, 1):
    col_type = "numeric" if df_main[col].dtype in ['int64', 'float64', 'Int64', 'Float64'] else "text"
    missing = df_main[col].isna().sum()
    print(f"  {idx}. {col} ({col_type}) - {missing} missing")

print("\n" + "="*70)
print("✅ NEW FEATURES ADDED SUCCESSFULLY!")
print("="*70)

📊 ADDING BUILDING & EARTHQUAKE DATA TO DATASET

📥 Loading new CSV files...
✅ Loaded buildings data: 959 rows, 9 columns
✅ Loaded earthquake data: 959 rows, 15 columns

📋 Buildings CSV columns:
  1. ilce_adi
  2. mahalle_adi
  3. mahalle_uavt
  4. 1980_oncesi
  5. 1980-2000_arasi
  6. 2000_sonrasi
  7. 1-4 kat_arasi
  8. 5-9 kat_arasi
  9. 9-19 kat_arasi

📋 Earthquake CSV columns:
  1. ilce_adi
  2. mahalle_adi
  3. mahalle_koy_uavt
  4. cok_agir_hasarli_bina_sayisi
  5. agir_hasarli_bina_sayisi
  6. orta_hasarli_bina_sayisi
  7. hafif_hasarli_bina_sayisi
  8. can_kaybi_sayisi
  9. agir_yarali_sayisi
  10. hastanede_tedavi_sayisi
  11. hafif_yarali_sayisi
  12. dogalgaz_boru_hasari
  13. icme_suyu_boru_hasari
  14. atik_su_boru_hasari
  15. gecici_barinma

📄 Sample buildings data (first 5 rows):
ilce_adi mahalle_adi  mahalle_uavt  1980_oncesi  1980-2000_arasi  2000_sonrasi  1-4 kat_arasi  5-9 kat_arasi  9-19 kat_arasi
  ADALAR   BURGAZADA         40139          433              214     

In [7]:
print("="*70)
print("📊 ADDING DISTRICT-LEVEL FEATURES TO DATASET")
print("="*70)

# Define district-level features
avg_rent_per_square = {
    "Bağcılar": 250, 
    "Bakırköy": 470, 
    "Beşiktaş": 560, 
    "Beykoz": 330, 
    "Fatih": 275,
    "Esenyurt": 200, 
    "Kadıköy": 550, 
    "Üsküdar": 550
}

avg_green_index = {
    "Bağcılar": 0.77, 
    "Bakırköy": 0.85, 
    "Beşiktaş": 0.93, 
    "Beykoz": 1, 
    "Fatih": 0.63,
    "Esenyurt": 0.57, 
    "Kadıköy": 0.84, 
    "Üsküdar": 0.88
}

avg_society_welfare = {
    "Bağcılar": 0.65, 
    "Bakırköy": 0.9, 
    "Beşiktaş": 1, 
    "Beykoz": 0.75, 
    "Fatih": 0.77,
    "Esenyurt": 0.4, 
    "Kadıköy": 0.85, 
    "Üsküdar": 0.80
}

print("\n📥 Loading current istanbul_mahalle_complete_data.csv...")
df_main = pd.read_csv('istanbul_mahalle_complete_data.csv')
print(f"✅ Loaded: {len(df_main)} rows, {len(df_main.columns)} columns")

# Add new columns based on İlçe (district)
print("\n🔗 Adding district-level features...")
df_main['Avg_Rent_Per_SqM'] = df_main['İlçe'].map(avg_rent_per_square)
df_main['Green_Index'] = df_main['İlçe'].map(avg_green_index)
df_main['Society_Welfare_Index'] = df_main['İlçe'].map(avg_society_welfare)

# Check matching
matched_rent = df_main['Avg_Rent_Per_SqM'].notna().sum()
matched_green = df_main['Green_Index'].notna().sum()
matched_welfare = df_main['Society_Welfare_Index'].notna().sum()

print(f"✅ Added Avg_Rent_Per_SqM: {matched_rent}/{len(df_main)} rows matched ({matched_rent/len(df_main)*100:.1f}%)")
print(f"✅ Added Green_Index: {matched_green}/{len(df_main)} rows matched ({matched_green/len(df_main)*100:.1f}%)")
print(f"✅ Added Society_Welfare_Index: {matched_welfare}/{len(df_main)} rows matched ({matched_welfare/len(df_main)*100:.1f}%)")

print("\n" + "="*70)
print("📊 UPDATED DATASET SUMMARY")
print("="*70)
print(f"\nTotal rows: {len(df_main)}")
print(f"Total columns: {len(df_main.columns)} (was {len(df_main.columns)-3}, added 3 new columns)")

print("\n📋 New columns added:")
print("  • Avg_Rent_Per_SqM - Average rent per square meter (TRY)")
print("  • Green_Index - Green space/environment index (0-1)")
print("  • Society_Welfare_Index - Social welfare index (0-1)")

print("\n" + "="*70)
print("📋 SAMPLE DATA WITH NEW FEATURES")
print("="*70)
sample_cols = ['Mahalle', 'İlçe', 'Avg_Rent_Per_SqM', 'Green_Index', 'Society_Welfare_Index']
print(df_main[sample_cols].head(10).to_string(index=False))

print("\n" + "="*70)
print("📊 DISTRICT-LEVEL STATISTICS")
print("="*70)
district_stats = df_main.groupby('İlçe')[['Avg_Rent_Per_SqM', 'Green_Index', 'Society_Welfare_Index']].first().sort_values('Avg_Rent_Per_SqM', ascending=False)
print(district_stats.to_string())

print("\n" + "="*70)
print("💾 SAVING UPDATED FILE")
print("="*70)
df_main.to_csv('istanbul_mahalle_complete_data.csv', index=False, encoding='utf-8-sig')
print(f"✅ Filename: istanbul_mahalle_complete_data.csv")
print(f"📊 Total rows: {len(df_main)}")
print(f"📊 Total columns: {len(df_main.columns)}")

print("\n" + "="*70)
print("✅ DISTRICT FEATURES ADDED SUCCESSFULLY!")
print("="*70)

📊 ADDING DISTRICT-LEVEL FEATURES TO DATASET

📥 Loading current istanbul_mahalle_complete_data.csv...
✅ Loaded: 164 rows, 46 columns

🔗 Adding district-level features...
✅ Added Avg_Rent_Per_SqM: 164/164 rows matched (100.0%)
✅ Added Green_Index: 164/164 rows matched (100.0%)
✅ Added Society_Welfare_Index: 164/164 rows matched (100.0%)

📊 UPDATED DATASET SUMMARY

Total rows: 164
Total columns: 49 (was 46, added 3 new columns)

📋 New columns added:
  • Avg_Rent_Per_SqM - Average rent per square meter (TRY)
  • Green_Index - Green space/environment index (0-1)
  • Society_Welfare_Index - Social welfare index (0-1)

📋 SAMPLE DATA WITH NEW FEATURES
  Mahalle     İlçe  Avg_Rent_Per_SqM  Green_Index  Society_Welfare_Index
 Balmumcu Beşiktaş               560         0.93                    1.0
    Bebek Beşiktaş               560         0.93                    1.0
   Kültür Beşiktaş               560         0.93                    1.0
Kuruçeşme Beşiktaş               560         0.93       